In [4]:
import pygplates
import pandas as pd
import numpy as np

from parameters import parameters as param

data = pd.read_csv('../data/db_ore_2.csv')

point_features = []
for index,row in data.iterrows():
    point = pygplates.PointOnSphere(float(row.lat),float(row.lon))
    point_feature = pygplates.Feature()
    point_feature.set_geometry(point)
    point_feature.set_name(str(index))
    point_features.append(point_feature)

static_polygons = pygplates.FeatureCollection(
    '../data/Shapefiles/StaticPolygons/Global_EarthByte_GPlates_PresentDay_StaticPlatePolygons_2015_v1.shp')
rotation_model = pygplates.RotationModel('../data/Global_EarthByte_230-0Ma_GK07_AREPS.rot')

# partition features
points = pygplates.partition_into_plates(static_polygons, rotation_model, point_features)
plate_ids = [p.get_reconstruction_plate_id() for p in points]
data['plate_id'] = plate_ids

#random ages
randomAges=np.random.randint(1, 230, size=data.shape[0])
data_random=[]
for index,row in data.iterrows():
    data_random.append([int(data.shape[0]+index+1), row.lon, row.lat, int(randomAges[index]), int(row.plate_id)])

data = data.append(pd.DataFrame(data_random, columns=data.columns))

#trench points
trench_points = np.loadtxt('./convergence_data/subStats_0.00.csv') #all subduction points at time 0    
trench_points=trench_points[(trench_points[:,9])==101] #subduction points in plate 101
trench_data=[]
i=0
for p in trench_points:
    for t in range(231):
        trench_data.append([int(data.shape[0]+i+1), p[0], p[1], int(t), int(p[9])]) 
        i+=1
        
data = data.append(pd.DataFrame(trench_data, columns=data.columns))

# data are ready and write them to file
with open('coregistration_input_data_Julian.csv',"w+") as f:
    for index,row in data.iterrows():
        f.write('{0:d}, {1:.2f}, {2:.2f}, {3:d}, {4:d}'.format(
            int(row.id),row.lon,row.lat,int(row.age),int(row.plate_id)))
        f.write('\n')
        
print('done!')

done!
